In [33]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers,Model

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

In [35]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [36]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download(['averaged_perceptron_tagger','stopwords','punkt','movie_reviews'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [7]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [5]:
df1=pd.read_excel("/content/drive/MyDrive/regresser/content_simulation_train.xlsx")
df1.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent


In [8]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
curr_time = pd.to_datetime("now")
df['diff_days']=curr_time-df['date']
df['diff_days']=df['diff_days']//np.timedelta64(1,'D')

In [9]:
df.head()

,id,date,likes,content,username,media,inferred company,diff_days
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,1097
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,1993
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,1171
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,1169
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,1882


In [7]:
df3=df1["likes"]

In [8]:
df3.head()

0       1
1    2750
2      57
3     152
4      41
Name: likes, dtype: int64

In [12]:
y=df.iloc[:,-1].values
y.shape

(300000,)

In [13]:
df.drop('likes',inplace=True,axis=1)

In [14]:
def extract_hashtags(x):
    x=x.strip()
    # print(x)
    hash_tags=[]
    x=x.split()
    for i in x:
        if len(i)>0:
            if i[0]=="#":
                hash_tags.append(i)
    k=",".join(hash_tags)
    k.replace("#","")
    return k
    # hashtags=[i for i in x.split(" ") if (len(i)>0) & (i[0]=="#")]
    # k=",".join(hashtags)
    # k.replace("#","")
    # # print(k)
    # # print(hashtags)
    # return k

In [15]:
df['hashtags']=df['content'].map(lambda x: extract_hashtags(x))
df.head()

,diff_days,inferred company,content,hashtags
0,1097,tim hortons,"Spend your weekend morning with a Ham, Egg, an...",#TimHortonsPH
1,1993,independent,Watch rapper <mention> freestyle for over an H...,
2,1171,cbc,Canadian Armenian community demands ban on mil...,
3,1169,williams,"1st in Europe to be devastated by COVID-19, It...",
4,1882,independent,Congratulations to Pauletha Butts of <mention>...,#excellenceisworththeeffort


In [4]:
!pip install wordninja
import wordninja

In [17]:
def cleaning_process(text1) :
    text = text1
    #print(text)
    if type(text) == str :
        text = re.sub(r"http\S+", "", text) # removing url

        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        #print(emoji_pattern.sub(r'', text)) # no emoji
        text = emoji_pattern.sub(r'', text)

        nopunc = [char for char in text if char not in string.punctuation] # removing punctuation
        text = ''.join(nopunc)

        text = ''.join([i for i in text if not i.isdigit()]) # removing digits

        text = text.lower()

        tokens = nltk.word_tokenize(text)
        tokens = [w for w in tokens if w not in stopwords.words('english')]
        #tokens = nltk.word_tokenize(text)

        #lematizer = WordNetLemmatizer()
        #ps = PorterStemmer()
        #lem_text = [lematizer.lemmatize(i) for i in tokens]
        #stem_text = [ps.stem(i) for i in lem_text]
        text = ' '.join(tokens)

        return text

    return None

In [18]:
df['clean_caption']=df['content'].apply(cleaning_process)
df['cleaned_hastags'] = df['hashtags'].apply(cleaning_process)

In [19]:
df['clean_caption'] = df['clean_caption'].apply(lambda x : ' '.join(wordninja.split(x)))
df['cleaned_hastags'] = df['cleaned_hastags'].apply(lambda x : ' '.join(wordninja.split(x)))

In [ ]:
df.to_csv("/content/drive/MyDrive/kaam_ka_folder/cleaned.csv")

In [9]:
df = pd.read_csv('/content/drive/MyDrive/regresser/cleaned.csv')
df.head()

,Unnamed: 0,diff_days,inferred company,content,hashtags,clean_caption,cleaned_hastags
0,0,1097,tim hortons,"Spend your weekend morning with a Ham, Egg, an...",#TimHortonsPH,spend weekend morning ham egg cheese wrap pair...,tim hortons ph
1,1,1993,independent,Watch rapper <mention> freestyle for over an H...,NaN,watch rapper mention freestyle hour hyperlink ...,NaN
2,2,1171,cbc,Canadian Armenian community demands ban on mil...,NaN,canadian armenian community demands ban milita...,NaN
3,3,1169,williams,"1st in Europe to be devastated by COVID-19, It...",NaN,st europe devastated c ovid italy redoubled ef...,NaN
4,4,1882,independent,Congratulations to Pauletha Butts of <mention>...,#excellenceisworththeeffort,congratulations paul eth a butts mention prese...,excellence is worth the effort


In [10]:
X1 = df['cleaned_hastags']
X1= df['cleaned_hastags'].fillna('')
tfidf = TfidfVectorizer(lowercase=False)
# tfidf = CountVectorizer()
text_vect = tfidf.fit_transform(X1)

In [11]:
words_idf_dict = dict()
words_list = tfidf.get_feature_names_out()
idf_list = tfidf.idf_
words_idf_dict = {words_list[i]:idf_list[i] for i in range(len(words_list))}

In [12]:
def get_idf_scores(text) :
    #print(text)
    t2 = nltk.word_tokenize(text)


    idf_list = [words_idf_dict[w] for w in t2 if len(w) > 1]
    avg_idf = np.mean(idf_list)

    return avg_idf

In [13]:
df['avg_idf'] = X1.apply(get_idf_scores)

In [14]:
df['length_hashtags'] = (df['hashtags'].fillna('')).apply(lambda x : x.count('#')) # no of topics
df['length_caption'] = df['clean_caption'].apply(lambda x : len(x.split(" "))) # no of words
# text polarity
from textblob import TextBlob
df['text_polarity'] = df['clean_caption'].apply(lambda x : TextBlob(x).sentiment[0])
df['text_subjectivity'] = df['clean_caption'].apply(lambda x : TextBlob(x).sentiment[1])
df.head()

,Unnamed: 0,diff_days,inferred company,content,hashtags,clean_caption,cleaned_hastags,avg_idf,length_hashtags,length_caption,text_polarity,text_subjectivity
0,0,1097,tim hortons,"Spend your weekend morning with a Ham, Egg, an...",#TimHortonsPH,spend weekend morning ham egg cheese wrap pair...,tim hortons ph,8.159732,1,22,0.175000,0.325000
1,1,1993,independent,Watch rapper <mention> freestyle for over an H...,NaN,watch rapper mention freestyle hour hyperlink ...,NaN,NaN,0,7,0.000000,0.000000
2,2,1171,cbc,Canadian Armenian community demands ban on mil...,NaN,canadian armenian community demands ban milita...,NaN,NaN,0,12,-0.100000,0.100000
3,3,1169,williams,"1st in Europe to be devastated by COVID-19, It...",NaN,st europe devastated c ovid italy redoubled ef...,NaN,NaN,0,14,0.500000,0.900000
4,4,1882,independent,Congratulations to Pauletha Butts of <mention>...,#excellenceisworththeeffort,congratulations paul eth a butts mention prese...,excellence is worth the effort,8.533557,1,23,0.133333,0.088889


In [15]:
df=df[["diff_days","inferred company","clean_caption","avg_idf","length_hashtags","length_caption","text_subjectivity",'text_polarity']]

In [16]:
df.head()

,diff_days,inferred company,clean_caption,avg_idf,length_hashtags,length_caption,text_subjectivity,text_polarity
0,1097,tim hortons,spend weekend morning ham egg cheese wrap pair...,8.159732,1,22,0.325000,0.175000
1,1993,independent,watch rapper mention freestyle hour hyperlink ...,NaN,0,7,0.000000,0.000000
2,1171,cbc,canadian armenian community demands ban milita...,NaN,0,12,0.100000,-0.100000
3,1169,williams,st europe devastated c ovid italy redoubled ef...,NaN,0,14,0.900000,0.500000
4,1882,independent,congratulations paul eth a butts mention prese...,8.533557,1,23,0.088889,0.133333


In [13]:
tfidf_wm = tfidf.fit_transform(df['clean_caption'])

In [18]:
tfidf_tokens = tfidf.get_feature_names_out()

In [ ]:
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)

In [17]:
tfidf_ic = tfidf.fit_transform(df['inferred company'])
tfidf_tokens = tfidf.get_feature_names_out()
df_tfidfvect1 = pd.DataFrame(data = tfidf_ic.toarray(),columns = tfidf_tokens)

In [18]:
df_tfidfvect1.head()

,aa,aaa,aarp,accenture,acer,aid,airasia,airlines,airtel,airways,...,westfield,wild,williams,wings,wireless,wolseley,wwf,yahoo,ymca,zumba
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df.drop(['inferred company', 'clean_caption'], axis=1)

In [19]:
df['avg_idf'] = df['avg_idf'].fillna(0)

In [20]:
result = pd.concat([df_tfidfvect1, df], axis=1, join='inner')

In [21]:
result.shape

(300000, 286)

In [ ]:
model=tf.keras.Sequential()

for i in range(0,5):
    if i==0:
        model.add(layers.Input(shape=(5,),batch_size=32))
        model.add(layers.Dense(64,activation='relu'))
    elif i%4==0:
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))
    else:
        model.add(layers.Dense(128,activation='relu'))

model.add(layers.Dense(64,activation='relu'))
# model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='linear'))
# model.load_weights('/kaggle/working/model.hdf5')
model.compile(optimizer="sgd",loss="mean_squared_error",metrics=['mae','accuracy'])
model.summary()

In [44]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger

# Tensorboard is a tool to monitor training.
tensorboard = TensorBoard(log_dir='/kaggle/working/tensorboard')

# Callback to save model after every epo/ch.
cp_callback = ModelCheckpoint(filepath='/kaggle/working/model.hdf5',
                             monitor='loss',
                             save_freq='epoch',
                             verbose=0,
                             period=1,
                             save_best_only=False,
                             save_weights_only=True)

# Keeps monitoring training and terminates it if model starts to overfit or value of specified monitoring metric is stationary.
es_callback = EarlyStopping(monitor='loss',
                            mode='min',
                            verbose=0,
                            patience=500)

# Dumps train and validation accuracy and loss to a csv file.
csv_logger = CSVLogger(filename='/kaggle/working/train_logs.csv',
                       separator=',',
                       append=False)

callbacks = [tensorboard, cp_callback, es_callback, csv_logger]

In [22]:
result.drop(['inferred company', 'clean_caption'], inplace=True,axis=1)
result.head()

,aa,aaa,aarp,accenture,acer,aid,airasia,airlines,airtel,airways,...,wwf,yahoo,ymca,zumba,diff_days,avg_idf,length_hashtags,length_caption,text_subjectivity,text_polarity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1097,8.159732,1,22,0.325000,0.175000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1993,0.000000,0,7,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1171,0.000000,0,12,0.100000,-0.100000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1169,0.000000,0,14,0.900000,0.500000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1882,8.533557,1,23,0.088889,0.133333


In [37]:
result = pd.read_csv("/kaggle/input/lv-dataset/result.csv")

In [38]:
df3 = pd.read_csv("/kaggle/input/lv-dataset/likes.csv",)

In [49]:
result.columns

Index(['aa', 'aaa', 'aarp', 'accenture', 'acer', 'aid', 'airasia', 'airlines',
       'airtel', 'airways',
       ...
       'wwf', 'yahoo', 'ymca', 'zumba', 'diff_days', 'avg_idf',
       'length_hashtags', 'length_caption', 'text_subjectivity',
       'text_polarity'],
      dtype='object', length=284)

In [14]:
df3.head()

,Unnamed: 0,likes
0,0,1
1,1,2750
2,2,57
3,3,152
4,4,41


In [39]:
y=df3["likes"].values
y.shape

(300000,)

In [ ]:
k = result.iloc[:,1:].values
k.shape
# y = df3.to_numpy()

In [ ]:
model.fit(k,y,epochs=100,batch_size=32, callbacks =callbacks)